#Causal Graph Fuzzy LLM (MISO)

In [ ]:
!pip install --quiet git+https://github.com/PatriciaLucas/AutoML.git
!pip install --quiet git+https://github.com/petroniocandido/clshq_tk.git
!git clone https://github.com/FutureLab-DCC/CGF-LLM.git

In [ ]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer
import torch
import sys
sys.path.append('CGF-LLM')
import CGFLLM, utils

In [ ]:
name_dataset = 'SONDA'
target = 'glo_avg'
df = utils.get_dataset(name = name_dataset)

max_lags = 20
partitions = 30
epochs = 20
database_metrics_path = 'CGFLLM_metrics.db'
database_size_path = 'CGFLLM_size.db'

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
name_model = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(name_model)

windows = utils.rolling_window(df, 10)

utils.execute("CREATE TABLE IF NOT EXISTS results(model TEXT, name_dataset TEXT, window INT, predict FLOAT, \
               real FLOAT)", database_metrics_path)
utils.execute("CREATE TABLE IF NOT EXISTS results(model TEXT, name_dataset TEXT, window INT, total_chars FLOAT, train_token_count FLOAT, test_token_count FLOAT, train_char_count FLOAT, \
               test_char_count FLOAT)", database_size_path)

In [ ]:
# Loop through the windows
for i, window in enumerate(windows):
      if i < 1:  # Enter the number of the last window saved in the database here
        print("window already executed")

      else:

        # Exclude constant series.
        for variable in window.columns:
            if window[variable].max() == window[variable].min():
                window = window.drop(variable, axis=1)
                print(f"Variables {variable} were deleted because they are constant.")

        # CAUSAL GRAPH FUZZY
        # model_name = 'CGF-LLM'
        # ds, scaler, tokenizer, seq, graph = CGFLLM.fuzzy_causal(window, name_dataset, target, max_lags, tokenizer, partitions)

        # TEXT
        # model_name = 'TEXT-LLM'
        # ds, scaler, tokenizer, seq, graph = CGFLLM.text(window, name_dataset, target, max_lags, tokenizer)

        # CAUSAL GRAPH TEXT
        # model_name = 'CGTEXT-LLM'
        # ds, scaler, tokenizer, seq, graph = CGFLLM.causal_text(window, name_dataset, target, max_lags, tokenizer)

        train_dataset, test_dataset = torch.utils.data.random_split(ds, [0.80, 0.20])

        freeze = True # Freeze GPT-2 weights
        output_size = 1
        model = CGFLLM.train_model(train_dataset, name_model, epochs, scaler, output_size, freeze)

        forecasts, real = CGFLLM.predict(test_dataset, model, tokenizer, target, scaler)

        for row in range(len(forecasts)):
            utils.execute_insert("INSERT INTO results VALUES(?, ?, ?, ?, ?)", (model_name, name_dataset, i, float(forecasts[row]), float(real[row])), database_metrics_path)

        # Calculates number of tokens and text
        total_chars, train_token_count, test_token_count, train_char_count, test_char_count = utils.calculate_train_test_sizes(ds, seq, tokenizer)
        utils.execute_insert("INSERT INTO results VALUES(?, ?, ?, ?, ?, ?, ?, ?)", (model_name, name_dataset, i, total_chars, train_token_count, test_token_count, train_char_count, test_char_count), database_size_path)

        print(f'Save window: {i}')

## How to view the results

In [ ]:
avg_metrics = utils.get_metrics(database_metrics_path)
avg_metrics

In [ ]:
avg_sizes = utils.get_sizes(database_size_path)
avg_sizes